# Preamble

In [74]:
from numpy import transpose, sum, array_equal, count_nonzero, zeros

# Shift Register Comparison

In [75]:
%cd /home/poltergeist/devel/src/mtt/case-studies/jon-cascade-03252019
from scipy.io import loadmat
from os.path import expanduser, join
from os import getcwd
import pandas as pd
import IPython
sr_matlab = zeros((350,4))
sr_matlab[:332,:] = loadmat('matlab-sr-15-4-10.mat')['prog_vect']
sr_mtt = loadmat('sr.mat')['prog_vect']
if sr_matlab.shape != sr_mtt.shape:
    print('\x1b[31mSHAPES ARE DIFFERENT!\x1b[0m')
    print('  {} vs {}'.format(sr_matlab.shape,sr_mtt.shape))

### Number of rows which are different

In [76]:
n_different_rows = count_nonzero(count_nonzero(sr_mtt - sr_matlab,axis=1))
if n_different_rows > 0:
    print('\x1b[31m{} different rows\x1b[0m'.format(n_different_rows))
else:
    print('Shift reg matrices are identical')

### Detail of rows that are different

In [77]:
# print(sr_matlab[280:287,:])
# print(sr_mtt[280:287,:])
for k,(row_mtt,row_matlab) in enumerate(zip(sr_mtt,sr_matlab)):
    if not array_equal(row_mtt,row_matlab):
        print('Row {} is different'.format(k))
        print('  {} vs {}'.format(row_mtt,row_matlab))

# SRAM Comparison

In [78]:
m_matlab = loadmat('matlab-sram-15-4-10.mat')['in_all']
m_mtt = loadmat('sram.mat')['in_all']

## Number of times each rule is used

The row sums (i.e. sum along each row) of these matrices should be either 100 (no wire is used for the rule) or 99 (one wire is used for the rule). A rule isn't used twice, so only one wire should be used per rule. However, each wire should be used twice (next section).

The row sums for each row should be equal (each rule was used the same number of times in both configs).

In [79]:
rowdiff = sum(m_mtt,1) - sum(m_matlab,1)
total_rowdiff = count_nonzero(rowdiff)
if total_rowdiff != 0:
    print('\x1b[31m{}\x1b[0m'.format('{} differences!'.format(total_rowdiff)))
else:
    print('Identical')

# Number of times each wire is used

Each wire should be used either zero times (1024 sum) or twice (1022 sum).

In [80]:
coldiff = sum(m_mtt,0) - sum(m_matlab,0)
total_coldiff = count_nonzero(coldiff)
if total_coldiff != 0:
    print('\x1b[31m{}\x1b[0m'.format('{} differences!'.format(total_coldiff)))
    IPython.display.display_html(pd.DataFrame(coldiff))
else:
    print('Identical')

In [81]:
from mtt import binaryToRuleCfg
for k in range(len(rowdiff)):
    if rowdiff[k] != 0:
        group, block, var = binaryToRuleCfg(k)
        if rowdiff[k] > 0:
            print("Matlab has a connection for group {} block {} var {} but mtt doesn't".format(group,block,var))
        if rowdiff[k] < 0:
            print("mtt has a connection for group {} block {} var {} but Matlab doesn't".format(group,block,var))

In [82]:
# compare rule connect_SRAM(in_all,0,0,12,0), wire 0
from mtt import SRAMRuleToBinary
print(m_mtt[SRAMRuleToBinary(0,1,11)])
print(m_matlab[SRAMRuleToBinary(0,1,11)])

In [83]:
bin(SRAMRuleToBinary(0,0,12))[2:].zfill(10)